In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# --- Leggi il dataset ---
df = pd.read_csv('../raw_datasets/middle_east_aggregated_data.csv', sep=';', encoding='utf-8')

# --- Parsing della data (mesi italiani → inglesi) ---
mesi = {
    'gennaio': 'January', 'febbraio': 'February', 'marzo': 'March',
    'aprile': 'April', 'maggio': 'May', 'giugno': 'June',
    'luglio': 'July', 'agosto': 'August', 'settembre': 'September',
    'ottobre': 'October', 'novembre': 'November', 'dicembre': 'December'
}

# Converte i mesi italiani per estrarre l'anno
df['WEEK_EN'] = df['WEEK'].replace(mesi, regex=True)
df['WEEK_DATE'] = pd.to_datetime(df['WEEK_EN'], errors='coerce', format='%d-%B-%Y')
df['YEAR'] = df['WEEK_DATE'].dt.year

NameError: name 'path' is not defined